In [1]:
# utilities
import re
import pickle
import numpy as np
import pandas as pd

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
from nltk.stem import WordNetLemmatizer

import tensorflow as tf


In [2]:
# Importing the dataset

item_table=pd.read_csv("/Volumes/Seagate/Datamyne Data/Export_00.csv",encoding = "ISO-8859-1", engine='c')
hscode_table=pd.read_csv("/Users/zhaomengshan/Desktop/Export_Import_project/harmonized-system-master/data/HS4.csv",encoding = "ISO-8859-1", engine='c')

In [4]:
import re
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return df

clean_text(item_table, 'Short Container Description', 'text_clean')

,Short Container Description,counter,text_clean
0,FAK,11673.0,fak
1,1 40' MILITARY DRY NO EEI PER 30.39 FREIGHT PR...,10320.0,military dry no eei per freight prepaid
2,AUTOMOBILE,8753.0,automobile
3,OFFICE AND LABORATORY SUPPLIES,6844.0,office and laboratory supplies
4,OFFICE AND LABORATORY SUPPLIES FREIGHT COLLECT,6616.0,office and laboratory supplies freight collect
...,...,...,...
1799678,1X40HC CONTAINER: PALLETS DRY - GDSM,1.0,xhc container pallets dry gdsm
1799679,1X40HC CONTAINER: PALLETS DRILLING FLUID ADDIT...,1.0,xhc container pallets drilling fluid additives...
1799680,1X40HC CONTAINER: PALLETS DOWNY ULTRA DOWNY AM...,1.0,xhc container pallets downy ultra downy amber ...
1799681,1X40HC CONTAINER: PALLETS DOWNY DWY LQ SA&RBLI...,1.0,xhc container pallets downy dwy lq sarblissz h...


In [5]:
clean_text(hscode_table, 'description', 'des_clean')

,ï»¿section,hscode,description,parent,level,des_clean
0,I,101,"Horses, asses, mules and hinnies; live",1,4,horses asses mules and hinnies live
1,I,102,Bovine animals; live,1,4,bovine animals live
2,I,103,Swine; live,1,4,swine live
3,I,104,Sheep and goats; live,1,4,sheep and goats live
4,I,105,"Poultry; live, fowls of the species Gallus dom...",1,4,poultry live fowls of the species gallus domes...
...,...,...,...,...,...,...
1218,XXI,9703,"Sculptures and statuary; original, in any mate...",97,4,sculptures and statuary original in any material
1219,XXI,9704,"Stamps, postage or revenue; stamp-postmarks, f...",97,4,stamps postage or revenue stamppostmarks first...
1220,XXI,9705,Collections and collectors' pieces; of zoologi...,97,4,collections and collectors pieces of zoologica...
1221,XXI,9706,Antiques; of an age exceeding one hundred years,97,4,antiques of an age exceeding one hundred years


In [ ]:
#input: hscode table, iterm table


#output item table, all possible pairs of (hscode + meaning)

#problem solving process: abstract to functions
#item table get key works

#build a dict key: all words, value: hscode
#example: "horse": 101, 103, 104

#use key word to find relavant hscode

#append relvant hscode+descrp to the item table


In [45]:
from typing import List



# get list of meaningful key words from a item table description col
#usage example:
#input:
#0	I	101	Horses, asses, mules and hinnies; live	1	4	horses asses mules and hinnies live
# output:
#["horses", "asses", "mules", "hinnies", "live"]

def get_key_word(item_table: pd.DataFrame) -> List[str]:
  #get des_clean
    row_id = item_table.index
    des_clean = item_table["text_clean"]
  #get list of words
    words_list = []
    for i in des_clean:
        a = i.split()
        words_list.append(a)
# I skip the remove meaningless words now, but it can be used if we want to remove certain words in the data, like and etc
    #meaningless_list = ["and"]
    #key_words_list = []
    #remove meaningless words
    #for words in key_words_list:
    #if word not in meaningless_list:
    #key_words_list.append(word_list)

    return row_id, words_list
    

In [46]:
get_key_word(item_table)

(RangeIndex(start=0, stop=1799683, step=1),
 [['fak'],
  ['military', 'dry', 'no', 'eei', 'per', 'freight', 'prepaid'],
  ['automobile'],
  ['office', 'and', 'laboratory', 'supplies'],
  ['office', 'and', 'laboratory', 'supplies', 'freight', 'collect'],
  ['consol'],
  ['waste', 'paperna'],
  ['grocery', 'store', 'merchandis'],
  ['plastic', 'scrapna'],
  ['automobiles'],
  ['autos'],
  ['mixed', 'metal', 'scrapna'],
  ['metal', 'scrapna'],
  ['military', 'reefer', 'no', 'eei', 'per', 'freight', 'prepaid'],
  ['synthetic', 'resin'],
  ['recovered', 'waste', 'and', 'scrap', 'pa'],
  ['military', 'dry', 'no', 'eei', 'per', 'freight', 'prepaid'],
  ['chemicals'],
  ['consolidated', 'cargo'],
  ['almonds'],
  ['major', 'movers'],
  ['metal', 'scrap'],
  ['hay', 'non', 'fumigated'],
  ['mixed', 'groceries'],
  ['office', 'and', 'laboratory', 'supplies', 'freight', 'prepaid'],
  ['household', 'goods'],
  ['grapes'],
  ['auto'],
  ['cleaning', 'and', 'personal', 'care', 'products'],
  ['resta

In [79]:
def build_hscode_dict(hscode_table: pd.DataFrame)-> dict :
#build a dict key: all words, value: hscode
# example: "horse": 101, 103, 104
# Get all the unique words in the HS4 dataset, and find corresponding HS code
# get the list of words

    hscode_dict = {}
    for row in (hscode_table):
        hscode = row["hscode"]
        des_clean = row["desc_clean"]
        word_list = []
        for i in des_clean:
            a = i.split()
            word_list.append(a)
            for word in word_list:
                if hscode_dict.get(word, None) is not None:
                    hscode_dict[word].append(hscode)
                else:
                    hscode_dict[word] = (hscode)
 
    return hscode_dict

In [65]:
#dict, search O(1), c++ 101, algorithm and design, O(n)
def find_possible_hscodes(words_list, hscode_dict) -> List[int]:
    hscode_list = []
  #get each line of key_words
    for word in list:
        if hscode_dict.get(word, None) is not None:
            hscode_list.append(hscode_dict[word])

    return row_id, hscode_list
   # Search it at the dictionary, if exist, return hs code, if not, print(row) return None

In [ ]:
find_possible_hscodes(words_list, hscode_dict)

In [69]:
def append_code_desc(item_table) -> None:
    hscode_dict = build_hscode_dict(hscode_table)
    for i in item_table:
        key_words_list = get_key_word(item_table)
    hscode_list = find_possible_hscodes(key_words_list, hscode_dict)
    if hscode_list.is_empty():
        print(row_id, item_table[row_id])
    else:
        for hscode in hscode_list:
            item_table.append(str(hscode) + get_desc_clean)
    return

In [ ]:
append_code_desc(item_table)